In [22]:
import pandas as pd
import numpy as np
from urllib.request import urlopen

In [2]:
k2 = pd.read_csv('/Users/ch/K2/projects/asteriks/asteriks/data/CAF_with_proposers.csv')

In [20]:
k2.head()

,obj_name,campaign,ap_start,ap_end,count,EPIC_ID,main_type,sub_type,new_obj_name,clean_name,alternate_names,PROPOSERS,IDS
0,Asteroid.Trojan.Leucus,111|112,200125327,200145290,2999,NaN,asteroid-Trojan,Leucus,Asteroid.Trojan.Leucus,Leucus,11351|1997 TS25|Leucus,Parker|Ryan,GO11100|GO11051
1,2013 A1 (Siding Springs),2,200001787,200004369,2583,NaN,comet,2013 A1 (Siding Springs),2013 A1 (Siding Springs),2013 A1 (Siding Springs),Siding Spring,Lisse|Kelley,GO2046|GO2030
2,Churyumov-Gerasimenko,10,200081804,200084003,2200,NaN,comet,67P/Churyumov-Gerasimenko,Churyumov-Gerasimenko,Churyumov-Gerasimenko,67P,Snodgrass,GO10904
3,Asteroid.Ryugu,15,200193072,200194787,1716,NaN,NaN,NaN,Asteroid.Ryugu,Ryugu,162173|Ryugu|1999 JU3,Kiss,GO15061
4,Asteroid.Trojan.2001 DU87,111|112,200113657,200134731,1700,NaN,asteroid-Trojan,2001 DU87,Asteroid.Trojan.2001 DU87,2001 DU87,2001 DU87|63291,Pal|Parker|Ryan,GO11114|GO11100|GO11051


In [11]:
piv = pd.read_pickle('all_piv.p')

In [18]:
piv.index.get_level_values('epoch').max()

2459035.5

In [21]:
HORIZONS_URL = ("https://ssd.jpl.nasa.gov/horizons_batch.cgi?"
                "batch=1&"
                "COMMAND=%27{0}%27&"
                "MAKE_EPHEM=%27YES%27%20&"
                "CENTER=%27500@-95%27&"
                "TABLE_TYPE=%27OBSERVER%27&"
                "START_TIME=%27{1}%27&"
                "STOP_TIME=%27{2}%27&"
                "STEP_SIZE=%27{3}%27%20&"
                "ANG_FORMAT=%27DEG%27&"
                "QUANTITIES=%271,3,9%27&"
                "CSV_FORMAT=%27YES%27".format('Leucus', 2458682.5, 2459035.5, 10))

In [35]:
res = str(urlopen(HORIZONS_URL).read())

In [42]:
inc = float(res.split('IN=')[-1].split('\\n')[0])
ecc = float(res.split('EC=')[-1].split('QR=')[0])
per = inc = float(res.split('IN=')[-1].split('\\n')[0])

!$$SOF
COMMAND= '11351;'
CENTER= '500@-95'
MAKE_EPHEM= 'YES'
TABLE_TYPE= 'OBSERVER'
START_TIME= '2019-03-12'
STOP_TIME= '2019-04-11'
STEP_SIZE= '1 d'
CAL_FORMAT= 'CAL'
TIME_DIGITS= 'MINUTES'
ANG_FORMAT= 'HMS'
OUT_UNITS= 'KM-S'
RANGE_UNITS= 'AU'
APPARENT= 'AIRLESS'
SUPPRESS_RANGE_RATE= 'NO'
SKIP_DAYLT= 'NO'
EXTRA_PREC= 'NO'
R_T_S_ONLY= 'NO'
REF_SYSTEM= 'J2000'
CSV_FORMAT= 'NO'
OBJ_DATA= 'YES'
QUANTITIES= '1,9,20,23,24'
!$$EOF